# Model Training and Registering

Este notebook está dedicado al entrenamiento de modelos de machine learning utilizando el dataset obtenido tras las tareas de preprocesamiento. El objetivo es resolver un problema cuya variable obejtivo es una variable categórica ordinal, por lo que se explorarán diferentes algoritmos para identificar el que mejor se adapte a nuestros datos.

En primer lugar, se propondrán tres modelos de clasificación: 
* LightGBM: Algoritmo basado en árboles de decisión optimizado para velocidad y eficiencia, especialmente útil en grandes conjuntos de datos y capaz de manejar variables categóricas y ordinales.
* XGBoost: Implementación avanzada de gradient boosting que destaca por su regularización y manejo eficiente de datos dispersos, logrando alto rendimiento en tareas de clasificación y regresión.
* CatBoosting: Algoritmo de boosting desarrollado por Yandex, diseñado para trabajar de forma nativa con variables categóricas y evitar el overfitting, ofreciendo excelentes resultados en problemas con datos heterogéneos.

Cada uno será entrenado y evaluado además se les dará seguimiento a sus experiment runs asociadas mediante MLflow para asegurar la trazabilidad de los resultados.

In [1]:
import mlflow
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor
import pandas as pd
from sklearn.metrics import cohen_kappa_score, root_mean_squared_error
import numpy as np
from sklearn.model_selection import train_test_split

**función auxiliar para logging**

In [2]:
import logging
import sys
from typing import Optional

def get_logger(name: str = __name__, log_file: Optional[str] = None, level=logging.INFO):
    """
    Create and configure a logger with optional file and console handlers.

    Args:
        name (str): Logger name (usually __name__).
        log_file (str, optional): Path to a file to log messages.
        level (int): Logging level (default: INFO).

    Returns:
        logging.Logger: Configured logger instance.
    """
    logger = logging.getLogger(name)
    logger.setLevel(level)

    if not logger.handlers:  # Prevent duplicate handlers in Jupyter or repeated calls
        formatter = logging.Formatter(
            fmt="[%(asctime)s | %(levelname)s ] %(name)s -> %(message)s",
            datefmt="%Y-%m-%d %H:%M:%S"
        )

        # Console handler
        console_handler = logging.StreamHandler(sys.stdout)
        console_handler.setFormatter(formatter)
        logger.addHandler(console_handler)

        # Optional file handler
        if log_file:
            file_handler = logging.FileHandler(log_file)
            file_handler.setFormatter(formatter)
            logger.addHandler(file_handler)

    return logger

In [3]:
# definimos el logger
logger = get_logger("mlflow experiments")

# Training Dataset

## Lectura del conjunto de entrenamiento

La variable objetivo, "Performance", es una variable categórica ordinal, lo que implica que sus categorías tienen un orden inherente pero no una distancia cuantificable entre ellas. Para abordar este tipo de problema, emplearemos algoritmos que permiten modelar la ordinalidad de la variable, como LightGBM, XGBoost y CatBoost. 

Es fundamental configurar correctamente los parámetros de estos modelos: 
 * en LightGBM se debe establecer el parámetro `objective` como `"multiclass"` o `"multiclassova"` y, para ordinalidad, considerar el uso de la variante `lgbm.rank` si se desea modelar el orden.
 * en XGBoost, el parámetro `objective` debe ser `"multi:softprob"` para clasificación multiclase; 
 * en CatBoost, se puede utilizar el parámetro `loss_function="MultiClass"` y, para ordinalidad, el modo `"YetiRank"` o `"Ordinal"` si se requiere modelar el orden explícitamente. 
 
 Además, es necesario especificar el número de clases (`num_class` o `classes_count`) y asegurarse de que la codificación de la variable objetivo respete el orden natural de las categorías. Estas configuraciones permiten que los modelos aprovechen la información ordinal y mejoren la capacidad predictiva en este contexto.

In [4]:
df = pd.read_csv("../data/processed/student_entry_performance_after_preprocessing.csv")
df.head()

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,Performance
0,0.803234,-0.597979,-0.285577,-0.770143,-0.017667,0.023372,-0.063243,-0.451023,-0.984059,-0.048342,-0.116927,0.258324,-0.415056,-0.299239,-0.045746,3
1,0.383741,-0.558693,0.664729,1.081237,1.002544,-0.000102,0.260944,0.398450,-0.258088,0.143168,0.352092,0.023940,0.116888,0.069941,-0.081184,3
2,0.665234,0.025918,0.910224,-0.979597,0.610582,0.515577,0.055653,0.029394,0.367403,0.276059,0.247405,0.025235,0.048601,0.496631,-0.060994,3
3,0.507289,-1.468973,-0.208351,0.967490,0.318739,-0.599155,0.271468,0.029258,0.281890,0.145417,0.155879,-0.050884,-0.074240,0.066675,-0.163159,3
4,0.676385,0.010505,0.672490,-0.781445,-0.002355,-0.421630,0.204497,0.090873,-0.031931,0.006785,-0.248873,-0.298813,-0.188266,0.556631,0.750617,3


## Separación en Conjunto de entrenamiento y prueba -estratificado-

In [5]:
target_column = 'Performance'

In [6]:
# X, matriz de caracteristicas
X = df.drop(columns=[target_column])
# y, vairable objetivo
y = df[target_column]

In [7]:
# Separación en conjunto de entrenamient y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=13,
    stratify=y
)

# Model training

El flujo básico de MLflow para la gestión de experimentos de machine learning consta de cuatro pasos fundamentales. 

**1. Definición del experimento:** Antes de iniciar cualquier entrenamiento, se debe crear o seleccionar un experimento en MLflow mediante `mlflow.create_experiment()` o `mlflow.set_experiment()` o `mlflow.get_experiment_by_name(experiment_name)`. Esto permite agrupar y organizar los diferentes intentos de entrenamiento bajo un mismo contexto, facilitando la trazabilidad y comparación de resultados. 

**2. Ejecución de una corrida (run) dentro del experimento:** Cada entrenamiento de modelo se encapsula en una "run", iniciada con `mlflow.start_run()`. Una run representa una ejecución individual, donde se pueden registrar todos los artefactos y métricas asociados. 

**3. Logging de parámetros, métricas y artefactos:** Durante la run, se emplean funciones como `mlflow.log_params()`, `mlflow.log_metrics()` y `mlflow.log_artifact()` para guardar los hiperparámetros utilizados, las métricas de desempeño obtenidas y cualquier archivo relevante (por ejemplo, el modelo entrenado o gráficos de evaluación). Este registro estructurado permite auditar y reproducir los experimentos fácilmente. 

**4. Registro del modelo:** Una vez identificado el mejor modelo, se utiliza `mlflow.register_model()` para almacenarlo en el Model Registry de MLflow. Esto habilita la gestión de versiones, la transición entre estados (staging, production, archived) y el despliegue controlado del modelo, asegurando que el ciclo de vida del modelo esté completamente documentado y gestionado. Este flujo garantiza la reproducibilidad, trazabilidad y gobernanza de los modelos desarrollados.


Es por ello que se ha decidido diseñar la función `evaluate_and_log_model()`. esta función trata de los siguiente:

Claro, aquí tienes una explicación paso a paso de lo que hace la función `evaluate_and_log_model`:

1. **Recibe los argumentos**:  
    - Nombre del modelo, instancia del modelo, datos de entrenamiento y prueba, parámetros, nombre del experimento MLflow y logger.

2. **Busca o crea el experimento en MLflow**:  
    - Si el experimento no existe, lo crea.  
    - Si ya existe, lo reutiliza.  
    - Informa por el logger qué experimento se está usando.

3. **Inicia una corrida (run) en MLflow**:  
    - Cada entrenamiento se encapsula en una run para registrar resultados.

4. **Loguea los parámetros del modelo**:  
    - Si se pasan parámetros, los registra en MLflow y lo informa por el logger.

5. **Entrena el modelo**:  
    - Ajusta el modelo con los datos de entrenamiento (`fit`).  
    - Informa por el logger que el entrenamiento terminó.

6. **Realiza predicciones**:  
    - Predice sobre el conjunto de prueba.  
    - Redondea las predicciones para obtener clases enteras.

7. **Calcula métricas**:  
    - Calcula el RMSE (error cuadrático medio raíz) entre las predicciones y los valores reales.  
    - Calcula el QWK (Cohen’s Kappa ponderado cuadrático) para evaluar la calidad de la clasificación ordinal.

8. **Registra las métricas en MLflow**:  
    - Loguea RMSE y QWK en MLflow.  
    - Informa por el logger los valores obtenidos.

9. **Guarda el modelo en MLflow**:  
    - Registra el modelo entrenado como artefacto en MLflow.  
    - Informa por el logger que el modelo fue guardado.

10. **Devuelve las métricas**:  
     - Retorna RMSE y QWK para su uso posterior.


**Métricas utilizadas: RMSE y QWK**

- **RMSE (Root Mean Squared Error):**  
    El RMSE es una métrica que mide la diferencia promedio entre los valores predichos por el modelo y los valores reales, penalizando más fuertemente los errores grandes. Se calcula como la raíz cuadrada de la media de los errores al cuadrado. En problemas de regresión y clasificación ordinal, el RMSE permite cuantificar qué tan cerca están las predicciones de los valores verdaderos, siendo útil para evaluar modelos que predicen valores continuos o clases ordenadas. Un RMSE bajo indica que el modelo realiza predicciones precisas.

- **QWK (Quadratic Weighted Kappa):**  
    El QWK es una métrica diseñada para evaluar la concordancia entre dos clasificadores (por ejemplo, las predicciones del modelo y las etiquetas reales) en problemas de clasificación ordinal. Considera no solo si la predicción es correcta, sino también cuán lejos está la predicción de la clase verdadera, penalizando más los errores grandes. El QWK toma valores entre -1 y 1, donde 1 indica perfecta concordancia, 0 indica concordancia aleatoria y valores negativos indican peor que aleatorio. Es especialmente relevante en contextos donde las clases tienen un orden natural, como en este caso.

Estas métricas permiten comparar objetivamente el desempeño de los modelos en la predicción de la variable ordinal "Performance", asegurando que tanto la precisión como el respeto por el orden de las categorías sean considerados.

## Función auxiliar para el logging con MLFlow

In [8]:
def evaluate_and_log_model(
    model_name,
    model,
    X_train,
    X_test,
    y_train,
    y_test,
    params=None,
    experiment_name="Default_Experiment",
    logger=None,
    tracking_dir="../data/mlflow"
):
    """
    Train model, log metrics, parameters, and model artifact to MLflow.
    Automatically creates or reuses an MLflow experiment by name.

    Args:
        model_name (str): Name of the model/run.
        model: Untrained model instance (e.g., sklearn model).
        X_train, X_test, y_train, y_test: Training/testing data.
        params (dict, optional): Model parameters to log.
        experiment_name (str): MLflow experiment name.
        logger (logging.Logger, optional): Logger for status messages.
        tracking_dir (str): Local path to store MLflow tracking data.
    """
    # --- Ensure MLflow uses the desired local folder ---
    mlflow.set_tracking_uri(f"file:{tracking_dir}")
    if logger:
        logger.info(f"MLflow tracking directory set to: {tracking_dir}")

    # --- Handle experiment setup ---
    existing_experiment = mlflow.get_experiment_by_name(name=experiment_name)
    if existing_experiment is None:
        experiment_id = mlflow.create_experiment(
            name=experiment_name,
            tags={"owner": "equipo36", "project": "student-performance-prediction"}
        )
        if logger:
            logger.info(f"Created new MLflow experiment: '{experiment_name}' (ID: {experiment_id})")
    else:
        experiment_id = existing_experiment.experiment_id
        if logger:
            logger.info(f"Using existing MLflow experiment: '{experiment_name}' (ID: {experiment_id})")

    # --- Start MLflow run ---
    with mlflow.start_run(experiment_id=experiment_id, run_name=model_name):
        if params:
            mlflow.log_params(params)
            if logger:
                logger.info(f"Logged parameters for {model_name}: {params}")

        # Fit model
        model.fit(X_train, y_train)
        if logger:
            logger.info(f"Model '{model_name}' training complete.")

        # Predict
        y_pred = model.predict(X_test)
        y_pred_class = np.rint(y_pred).astype(int)

        # Compute metrics
        rmse = root_mean_squared_error(y_test, y_pred_class)
        qwk = cohen_kappa_score(y_test, y_pred_class, weights="quadratic")

        # Log metrics
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("quadratic_weighted_kappa", qwk)
        if logger:
            logger.info(f"Metrics logged — RMSE: {rmse:.4f}, QWK: {qwk:.4f}")

        # Log the model
        mlflow.sklearn.log_model(model, artifact_path="model", input_example=X_test.iloc[:5])
        if logger:
            logger.info(f"Model '{model_name}' logged to MLflow under experiment '{experiment_name}'.")

        return rmse, qwk

## MLFlow server

MLflow server es una herramienta que permite gestionar y monitorizar experimentos de machine learning de forma centralizada. Su propósito principal es almacenar los resultados, parámetros, métricas y modelos generados durante el ciclo de vida de los experimentos, facilitando la trazabilidad, comparación y reproducibilidad. Al ejecutar el servidor MLflow, se habilita una interfaz web donde los usuarios pueden visualizar y administrar todos los experimentos registrados en la plataforma.

En la celda siguiente, podemos encontrar código que nos ayuda a levantar el server.

In [9]:
import subprocess, time, os, signal

# Configuration
MLFLOW_HOST = "127.0.0.1"
MLFLOW_PORT = 8080

# Start the MLflow server in background
mlflow_process = subprocess.Popen(
    [
        "mlflow", "server",
        "--host", MLFLOW_HOST,
        "--port", str(MLFLOW_PORT),
    ],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True
)

print(f"✅ MLflow server started at http://{MLFLOW_HOST}:{MLFLOW_PORT}")
time.sleep(3)

✅ MLflow server started at http://127.0.0.1:8080


## LightGBM



### Definición

In [10]:
# Define LightGBM params
params_lgb = {
    'objective': 'regression',    # or 'rmse' — continuous ordinal target
    'metric': 'rmse',
    'learning_rate': 0.05,
    'num_leaves': 31,
    'max_depth': -1,
    'min_data_in_leaf': 20,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': -1,
    'random_state': 13
}

# Initialize model
model_lgb = lgb.LGBMRegressor(**params_lgb)

### Entrenamiento y logging con mlflow

In [11]:
lgb_rmse, lgb_qwk = evaluate_and_log_model(
    experiment_name="mlflow-student-performance-experiment",
    model_name="LightGBM",
    model=model_lgb,
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    y_test=y_test,
    params=params_lgb,
    logger=logger
)

[2025-10-12 18:05:06 | INFO ] mlflow experiments -> MLflow tracking directory set to: ../data/mlflow
[2025-10-12 18:05:06 | INFO ] mlflow experiments -> Created new MLflow experiment: 'mlflow-student-performance-experiment' (ID: 155893811526728657)
[2025-10-12 18:05:06 | INFO ] mlflow experiments -> Logged parameters for LightGBM: {'objective': 'regression', 'metric': 'rmse', 'learning_rate': 0.05, 'num_leaves': 31, 'max_depth': -1, 'min_data_in_leaf': 20, 'feature_fraction': 0.8, 'bagging_fraction': 0.8, 'bagging_freq': 5, 'verbose': -1, 'random_state': 13}
[2025-10-12 18:05:08 | INFO ] mlflow experiments -> Model 'LightGBM' training complete.
[2025-10-12 18:05:08 | INFO ] mlflow experiments -> Metrics logged — RMSE: 0.8819, QWK: 0.5097


2025/10/12 18:05:08 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
c:\Users\81083075\Documents\ML Projects\MLOps\equipo36mlops\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-10-12 18:05:30 | INFO ] mlflow experiments -> Model 'LightGBM' logged to MLflow under experiment 'mlflow-student-performance-experiment'.


## XGBoost

### Definición

In [12]:
# Definición del modelo
# NOTE: Usamos un diccionario para definir los hiperparámetros del modelo
params_xgb = {
    'objective': 'reg:squarederror',
    'n_estimators': 200,
    'learning_rate': 0.05,
    'max_depth': 6,
    'random_state': 13
}
# Instanciamos el modelo
model_xgb = xgb.XGBRegressor(**params_xgb)

### Entrenamiento y loggeo con mlflow

In [13]:
xgb_rmse, xgb_qwk = evaluate_and_log_model(
    experiment_name="mlflow-student-performance-experiment",
    model_name="XGBoost",
    model=model_xgb,
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    y_test=y_test,
    params=params_xgb,
    logger=logger
)

[2025-10-12 18:05:30 | INFO ] mlflow experiments -> MLflow tracking directory set to: ../data/mlflow
[2025-10-12 18:05:30 | INFO ] mlflow experiments -> Using existing MLflow experiment: 'mlflow-student-performance-experiment' (ID: 155893811526728657)
[2025-10-12 18:05:30 | INFO ] mlflow experiments -> Logged parameters for XGBoost: {'objective': 'reg:squarederror', 'n_estimators': 200, 'learning_rate': 0.05, 'max_depth': 6, 'random_state': 13}
[2025-10-12 18:05:32 | INFO ] mlflow experiments -> Model 'XGBoost' training complete.
[2025-10-12 18:05:32 | INFO ] mlflow experiments -> Metrics logged — RMSE: 0.8477, QWK: 0.5524


2025/10/12 18:05:32 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


[2025-10-12 18:05:37 | INFO ] mlflow experiments -> Model 'XGBoost' logged to MLflow under experiment 'mlflow-student-performance-experiment'.


## CatBoosting

### Definición del modelo

In [14]:
# Define CatBoost parameters
params_cat = {
    'iterations': 300,
    'learning_rate': 0.05,
    'depth': 6,
    'loss_function': 'RMSE',   # Continuous target, respects ordering
    'random_seed': 42,
    'verbose': 0,
    'od_type': 'Iter',         # Enable early stopping
    'od_wait': 20
}

# Initialize model
model_cat = CatBoostRegressor(**params_cat)

### Entrenamiento y loggeo con mlflow

In [15]:
cat_rmse, cat_qwk = evaluate_and_log_model(
    experiment_name="mlflow-student-performance-experiment",
    model_name="CatBoost",
    model=model_cat,
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    y_test=y_test,
    params=params_cat,
    logger=logger
)

[2025-10-12 18:05:37 | INFO ] mlflow experiments -> MLflow tracking directory set to: ../data/mlflow
[2025-10-12 18:05:37 | INFO ] mlflow experiments -> Using existing MLflow experiment: 'mlflow-student-performance-experiment' (ID: 155893811526728657)
[2025-10-12 18:05:37 | INFO ] mlflow experiments -> Logged parameters for CatBoost: {'iterations': 300, 'learning_rate': 0.05, 'depth': 6, 'loss_function': 'RMSE', 'random_seed': 42, 'verbose': 0, 'od_type': 'Iter', 'od_wait': 20}
[2025-10-12 18:05:38 | INFO ] mlflow experiments -> Model 'CatBoost' training complete.
[2025-10-12 18:05:38 | INFO ] mlflow experiments -> Metrics logged — RMSE: 0.9108, QWK: 0.4491


2025/10/12 18:05:38 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


[2025-10-12 18:05:42 | INFO ] mlflow experiments -> Model 'CatBoost' logged to MLflow under experiment 'mlflow-student-performance-experiment'.


# Finalmente

Detenemos el servidor de MLFlow

In [16]:
# if 'mlflow_process' in locals() and mlflow_process.poll() is None:
#     os.kill(mlflow_process.pid, signal.SIGTERM)
#     print("🛑 MLflow server stopped.")
# else:
#     print("MLflow server was not running or already stopped.")